In [1]:
import win32com.client as win32
import numpy as np
from matplotlib import cm
import pywintypes
import os

### Defining of required functions:
(see their usage below)

In [2]:
def add_sphere(mws, name, component, material, center_radius, center):
    """
    This function adds sphere in the CST project
    arguments:
    mws - project object
    name - name of sphere
    component - create or set an existing component in CST Navigation tree
    material - sets the material for sphere
    center_radius - RADIUS of the sphere
    center - COORDINATES of the center
    """
    sp = mws.Sphere  # call Sphere method. You can see a CST VBA documentation to learn more about all methods
    sp.Reset()
    sp.Name(name)
    sp.Component(component)
    sp.Material(material)
    sp.CenterRadius(center_radius)  # must be an Integer
    sp.Center(str(center[0, 0]), str(center[0, 1]), str(center[0, 2]))  # must be a strings
    sp.Segments("0")  # here you can create interesting polyhedra with values of 3-10
    sp.Create()


def add_material(mws, mat_name, epsilon, colors):
    """
    This functions creates a new material in the CST project
    :param mws: project object
    :param mat_name: name of the material
    :param epsilon: permittivity
    :param colors: RGB-triplet corresponding to epsilon value
    """
    mat = mws.Material
    mat.Name(mat_name)
    mat.FrqType('all')
    mat.Type('Normal')
    mat.Sigma('0.005')
    mat.SetMaterialUnit('GHz', 'mm')
    mat.Epsilon(str(epsilon))
    mat.Mue('1.0')
    mat.DynamicViscosity('0')
    mat.TanD('0.0')
    mat.TanDFreq('0.0')
    mat.KappaM('0')
    mat.TanDM('0.0')
    mat.TanDMFreq('0.0')
    mat.TanDMGiven('False')
    mat.DispModelEps('None')
    mat.DispModelMue('None')
    mat.UseGeneralDispersionEps('False')
    mat.Rho('0')
    mat.ThermalType('Normal')
    mat.ThermalConductivity('0')
    mat.HeatCapacity('0')
    mat.SetActiveMaterial('all')
    mat.Colour(str(colors[epsilon-1, 0]), str(colors[epsilon-1, 1]), str(colors[epsilon-1, 2]))
    mat.Wireframe('False')
    mat.Transparency('50')
    mat.Create()


def insert(mws, a, b):
    """
    This function helps to insert object B in the object A
    :param mws: project object
    :param a: will be cutted for the shape of "b"
    :param b: what to insert in a
    """
    sol = mws.Solid
    sol.Insert(a, b)


# Start Point:

Here we automatically open a CST app and get __the object__ which corresponds to our __"test_inhomogeneities.cst"__ file in MWS variable:

Now we can call any available method from CST API to build or automate our CST project

In [3]:
path_to_this_folder = os.path.dirname(os.path.abspath("main.py"))  # auto path
cst = win32.gencache.EnsureDispatch('CSTStudio.Application')  # connect to CST API and open the application
mws = cst.OpenFile(path_to_this_folder + "\\test_inhomogeneities.cst")  # open our test CST file. 
# Dont open it manually, it will be an error
# MWS will be our main object to work with.

In the cell below you can define:
+ inclusion sizes 
+ number of inclussions
+ material (a range of permittivity)
+ borders for random filling 

Any other parameter is very easy to define. See _CST Studio Suite Help\Automation and scripting\VBA_

You will get this result:

![gif1](readme_files/filling.gif)

In [4]:
max_radius = 15  # maximum radius of inserting element
min_radius = 5  # min radius
number_of_balls = 1000  # number of inserting elements

# In this project I want to fill my volume with elements with random distribution of permittivity in range 1-40
max_epsilon = 40
# and for the more clearness I want to highlight all the elements with
# some colormap in dependence of value of their of permittivity
viridis = cm.get_cmap('viridis')  # get a colormap instance
colors = viridis(np.linspace(0, 1, max_epsilon))  # using CST API you can set colors in [R, G, B] format
# so I need to obtain "max_epsilon" values of colors in range 0-1
colors = colors[:, 0:3]  # here we don't need 4th alpha-channel
# thus we created our own colormap for labeling of permittivity of objects

# At first we create 40 materials corresponding to our permittivity
for i in range(0, max_epsilon, 1):
    material = 'mat' + str(i + 1)  # define the name of material
    epsilon = i + 1  # set its permittivity (there is no zero permittivity in physics)
    add_material(mws, material, int(epsilon), colors)  # add this material to the CST project
    # see how to set any other parameter in "add_material" function or in CST VBA documentation

all_centers = np.zeros(shape=(number_of_balls, 3))  # zero array for saving the center positions of all inserting objects
# in this loop we will add objects in a random positions in our volume and
# randomly will define a material for every object.
for elem in range(0, number_of_balls):
    # Here we set random X , Y and Z coordinates for the center of every object
    center_radius = np.random.randint(min_radius, max_radius, 1)  # random radius
    x_center = np.random.randint((-500 + max_radius + 1), (500 - max_radius - 1), 1)
    y_center = np.random.randint((-500 + max_radius + 1), (500 - max_radius - 1), 1)
    z_center = np.random.randint((-500 + max_radius + 1), (0 - max_radius - 1), 1)
    # the object will not touch the borders of inserting volume
    # these values correspond to the size of the ground brick
    Center = np.transpose(np.array([x_center, y_center, z_center]))
    all_centers[elem, :] = Center  # save object center coorditanes in a zero array

    # Define object name and its random material
    component = 'adds'  # name of the component of inserts in the Navigation Tree
    Name = 'Sphere' + str(elem)  # name of the sphere (define the number os segments in "add_sphere" first.
    # The more the segments the more time you need to generate them. Sphere with 3-10 segments can serve as polyhedron)
    rand_mat = np.random.randint(1, 41, 1)  # randomly chose a number in range 1-40
    material = 'mat' + str(rand_mat[0])  # here we randomly picking one of 40 materials

    add_sphere(mws, Name, component, material, int(center_radius), Center)  # add sphere to the volume of ground

    # also a new object need to be inserted to the ground medium
    insert_name = 'adds:Sphere' + str(elem)  # set a full path to the object in Navigation Tree
    insert(mws, 'component1:ground_brick', insert_name)  # here we use build-in method from CST API

### Objects overlapping solving algorithm
if you have created a big number of inclusions, you will definitely get an object overlapping. It may cause inaccurate results after electromagnetic simulation. 

>Your CST project must have no intersection problems for acurate results.

Here what this algorithm does on the example with 2 spheres:

Before:

![overlapping](readme_files/overlapped_fig.png)

After:

![resolve overlapping](readme_files/resolved_overlapping.png)

In [5]:
# Algorithm of searching and inserting of overlapped objects
# unfortunately CST API does not support "Intersection Check" method from the Simulation tab
# So I've made similar algorithm by myself:
for j in range(0, number_of_balls):  # pick a one object (its radius)
    for k in range(0, number_of_balls):  # compare this radius with radii of every other object
        r = ((all_centers[j, 0] - all_centers[k, 0]) ** 2 + (all_centers[j, 1] - all_centers[k, 1]) ** 2 + (
                all_centers[j, 2] - all_centers[k, 2]) ** 2) ** 0.5  # find a distance between centers of these objects
        # see a formula in README.md
        if 0 < r < (2 * max_radius):  # if this distance will be smaller than maximum double radius
            # (2 biggest possible spheres) - they are 100% overlapped
            # of course this rule is not perfect and captures a smaller spheres, BUT:
            # 1) this method is much faster than total self-insert of all objects
            # 2) All overlapped conflicts will be resolved
            sphere_j = 'adds:Sphere' + str(j)  # choose a verifiable sphere
            sphere_k = 'adds:Sphere' + str(k)  # choose an overlapped sphere
            # but before insert there is one important moment
            # if you have a big range of radii, smaller object can appear inside of the bigger objects
            # and after inserting, smaller object is totally disappears from the project
            # but its radius is still in our array of radii ("all_centers").
            # and when you try to insert a non-existing element of course you will get an error.
            # here it is perfect place to use try-except expression
            try:
                insert(mws, sphere_j, sphere_k)
            except pywintypes.com_error:
                continue  # in case of absence of object - just continue the loop

### Mine and objects overlapping solving
In the same way we can resolve overlapping for any object. This is how it looks for PMN-4 mine as an example:

![resolve overlapping with mine](readme_files/pmn4_overlapping_resolve.png)

In [6]:
mine_center = np.array([0, 0, -30])  # define Mine center
# (by the way you can find "Picks" methods in CST API documentation to do it automatically in your project)
for k in range(0, number_of_balls):
    r = ((mine_center[0] - all_centers[k, 0]) ** 2 +
         (mine_center[1] - all_centers[k, 1]) ** 2 +
         (mine_center[2] - all_centers[k, 2]) ** 2) ** 0.5  # find a distance between mine center and object
    if 0 < r < (max_radius + 80):  # here we mentally put out mine into sphere of its radius to create
        # something like "inviolability zone", from witch inclusion objects must be intersected
        # mine radius is about 60mm. Mine shape is not perfect sphere and lets add some
        # reserve distance (10-20mm in my case). Be careful to take into account your objects shape.
        # We apply the same rule like in the first algorithm
        sphere_k = 'adds:Sphere' + str(k)  # catch an object in "inviolability zone"
        # and for the same reason like in the previous algorithm wee need try-except expression
        try:
            insert(mws, sphere_k, 'PMN:air')  # Here we have to insert all the inner parts of mine
            insert(mws, sphere_k, 'PMN:body')
            insert(mws, sphere_k, 'PMN:explosive')
            insert(mws, sphere_k, 'PMN:metal')
            insert(mws, sphere_k, 'PMN:rubber')
        except pywintypes.com_error:
            continue


### I hope this project was usefull for you, Enjoy!